In [ ]:
import pandas as pd
from glob import glob
import os

In [ ]:
root_folder = "Synthetic datasets"
baseline_folder=os.path.join(root_folder, "baseline")
results_folder=os.path.join(root_folder, "results")
constraint_folder = os.path.join(root_folder, "constraints")

In [ ]:
scores_df = pd.concat([pd.read_csv(result_file) for result_file in glob(os.path.join(results_folder,"*.csv"))], ignore_index=True)

In [ ]:
# To allow comparison of the case where all models are blended together, we must add the "mix" case to the 
# baseline scores
baseline_df = pd.concat([pd.read_csv(baseline_file) for baseline_file in glob(os.path.join(baseline_folder,"*.csv"))], ignore_index=True)
baseline_df = baseline_df[baseline_df.Score!="ARI"]
mix_baseline_df= baseline_df.groupby(["Dataset","Model"],as_index=False).Run.nunique()
mix_baseline_df["Run"] = mix_baseline_df.groupby(["Dataset"])["Run"].cumsum()-mix_baseline_df["Run"]
mix_baseline_df=pd.merge(baseline_df, mix_baseline_df.rename(columns={"Run":"Offset"}), on=["Dataset","Model"], how="left")
mix_baseline_df["Run"] = mix_baseline_df["Run"]+mix_baseline_df["Offset"]
mix_baseline_df["Model"] = "mix"
baseline_df = pd.concat([baseline_df, mix_baseline_df.drop("Offset", axis=1)], axis=0, ignore_index=True)

In [ ]:
df=pd.concat([baseline_df, scores_df], axis=0, ignore_index=True)
constraints_df = pd.concat([pd.read_csv(constraint_file) for constraint_file in glob(os.path.join(constraint_folder, "*.csv"))], ignore_index = True)

In [ ]:
def compute_correlation_table(df, index=["Dataset"], target="ARI", correlation_method="pearson"):
    # First, we pivot the dataset to get all the scores on the columns (ie as variables)
    pivoted_df = df.pivot(columns="Score",index=["Dataset","Run","Model"], values="Value").reset_index()

    # Then, we drop irrelevant columns and perform correlation after a grouping operation
    columns_to_drop = [x for x in df.columns if x not in index+[target, "Score", "Value"]]
    df = pivoted_df.drop(columns_to_drop, axis=1).groupby(index).corr(method=correlation_method)

    # Then, we can pivot the dataframe to get all different scores on the columns
    df = df[target].reset_index().rename(columns={target:"Correlation"})
    df = df.pivot(columns="Score", index=index, values="Correlation")

    # We do not forget to drop the column containing the target (correlation with itself=1)
    df=df.drop(target, axis=1)

    return df.reset_index()

In [ ]:
def compute_target_table(df, index=["Dataset"], target="ARI"):
    # We must first reduce the dataframe to the best model according to each metric
    best_df = df.loc[df.groupby(index+["Score"])["Value"].idxmax().dropna()].drop("Value",axis=1)
    target_df = df[df.Score==target].drop("Score", axis=1)

    # We can then merge the dataframe with itself to get the corresponding target value
    keys = [x for x in df.columns if x not in ["Score","Value"]]
    best_df = pd.merge(best_df, target_df)

    # We can then pivot the dataframe while tossing away irrelevant identifying columns
    columns_to_drop = [x for x in df.columns if x not in index+[target, "Score", "Value"]]
    best_df = best_df.drop(columns_to_drop,axis=1)
    pivoted_df = best_df.pivot(columns="Score",index=index, values="Value").reset_index()

    # We drop the target variable
    pivoted_df = pivoted_df.drop(target, axis=1)

    return pivoted_df   

In [ ]:
def compute_regret(df, index=["Dataset"]):
    # We simply identify the best value per row
    score_columns = [x for x in df.columns if x not in index]
    best_scores = df[score_columns].max(1)

    # And then we subtract (notice the - to get "max - value")
    regret_scores = -df[score_columns].sub(best_scores, axis=0)
    regret_scores[index] = df[index]

    return regret_scores

In [ ]:
def aggregate_table(df):
    if "Dataset" in df.columns:
        df = df.drop("Dataset", axis=1)
    aggregator = lambda x: f"{x.mean():.2f}"+"\\std{"+f"{x.std():.2f}"+"}"
    return df.groupby("Model").agg(aggregator)

# Correlations

## Pearson

In [ ]:
# First table: the correlation
correlation_df = compute_correlation_table(df, index=["Dataset", "Model"], correlation_method="pearson")
result = aggregate_table(correlation_df)
display(result.T)
print(result.T.to_latex())

# Then, we can compute the regret on this correlation
correlation_regret = compute_regret(correlation_df, index=["Dataset", "Model"])
result = aggregate_table(correlation_regret)
display(result.T)
print(result.T.to_latex())

## Kendall

In [ ]:
# First table: the correlation
correlation_df = compute_correlation_table(df, index=["Dataset", "Model"], correlation_method="kendall")
result = aggregate_table(correlation_df)
display(result.T)
print(result.T.to_latex())

# Then, we can compute the regret on this correlation
correlation_regret = compute_regret(correlation_df, index=["Dataset", "Model"])
result = aggregate_table(correlation_regret)
display(result.T)
print(result.T.to_latex())

# ARI of best model per score

In [ ]:
best_df = compute_target_table(df, index=["Dataset", "Model"])

# Then, we can compute the regret on this correlation
best_regret = compute_regret(best_df, index=["Dataset", "Model"])
result = aggregate_table(best_regret)
display(result.T)
print(result.T.to_latex())